In [1]:
import cv2
import numpy as np
import scipy
import matplotlib.pyplot as plt
import mahotas as mt
import os, os.path
import _pickle as cPickle
import re
import pandas as pd
import warnings
from scipy.ndimage import measurements
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, train_test_split, cross_val_score , ShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from ast import literal_eval
from itertools import combinations
import h5py


import sys
sys.path.append('./texture_analysis')
from TextureAnalysis import GLCM_3D, GLSZM_3D, NGTDM, NGTDM_3D, GLHA
from ipywidgets import interact
from IPython.core.pylabtools import figsize
from IPython.display import display
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [2]:
def extract_haralick_features(image_path):
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    # calculate haralick texture features for 4 types of adjacency
    haralick = mt.features.haralick(image).mean(axis=0)
    # take the mean of it and return it
    return haralick

def extract_lbp_features(image_path):
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    # calculate lbp texture features (image, radius, points, ignore_zeros=False)
    lbp_textures = mt.features.lbp(image, 8, 8, False)
    return lbp_textures

def extract_histogram_features(image_path):
    # convert the image to HSV color-space
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hHistr = list(cv2.calcHist([image],[0],None,[32],[0,180]).flatten())
    sHistr = list(cv2.calcHist([image],[1],None,[16],[0,256]).flatten())
    vHistr = list(cv2.calcHist([image],[2],None,[16],[0,256]).flatten())
#     hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 180, 0, 256, 0, 256])
    hist = np.concatenate((hHistr,sHistr,vHistr))
    return np.array(hist)

In [3]:
def extract_correlogram_features(image_path):
    image = cv2.imread(image_path)
    Z = image.reshape((-1, 3))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 16
    ret, label, center = cv2.kmeans(Z, K, None,criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))
    # according to "Image Indexing Using Color Correlograms" paper
    K = [i for i in range(1, 9, 2)]
    colors64 = unique(np.array(res))
    result = np.array(correlogram(res2, colors64, K))
    return result.flatten()

def correlogram(image, Cm, K):
    X, Y, t = image.shape
    colorsPercent = []
    for k in K:
        countColor = 0
        color = []
        for i in Cm:
            color.append(0)
        for x in range(0, X, int(round(X / 10))):
            for y in range(0, Y, int(round(Y / 10))):
                Ci = image[x][y]
                Cn = getNeighbors(X, Y, x, y, k)
                for j in Cn:
                    Cj = image[j[0]][j[1]]
                    for m in range(len(Cm)):
                        if np.array_equal(Cm[m], Ci) and np.array_equal(Cm[m], Cj):
                            countColor = countColor + 1
                            color[m] = color[m] + 1
        for i in range(len(color)):
            color[i] = float(color[i]) / countColor
        colorsPercent.append(color)
    return colorsPercent

def unique(a):
    #remove duplicates from input list
    order = np.lexsort(a.T)
    a = a[order]
    diff = np.diff(a, axis = 0)
    ui = np.ones(len(a), 'bool')
    ui[1:] = (diff != 0).any(axis = 1)
    return a[ui]

def getNeighbors(X, Y, x, y, dist):
    cn1 = (x + dist, y + dist)
    cn2 = (x + dist, y)
    cn3 = (x + dist, y - dist)
    cn4 = (x, y - dist)
    cn5 = (x - dist, y - dist)
    cn6 = (x - dist, y)
    cn7 = (x - dist, y + dist)
    cn8 = (x, y + dist)
    points = (cn1, cn2, cn3, cn4, cn5, cn6, cn7, cn8)
    Cn = []
    for i in points:
        if isValid(X, Y, i):
            Cn.append(i)
    return Cn

def isValid(X, Y, point):
    if point[0] < 0 or point[0] >= X:
        return False
    if point[1] < 0 or point[1] >= Y:
        return False
    return True

In [4]:
def extract_cld_features(image_path):
    image = cv2.imread(image_path)
    cld_rows = 4
    cld_cols = 4
    averages = np.zeros((cld_rows,cld_cols,3))
    imgH, imgW, _ = image.shape
    for row in range(cld_rows):
        for col in range(cld_cols):   
            slice = image[int(imgH/cld_rows * row): int(imgH/cld_rows * (row+1)),int(imgW/cld_cols*col) : int(imgW/cld_cols*(col+1))]
            average_color_per_row = np.mean(slice, axis=0)
            average_color = np.mean(average_color_per_row, axis=0)
            average_color = np.uint8(average_color)
            averages[row][col][0] = average_color[0]
            averages[row][col][1] = average_color[1]
            averages[row][col][2] = average_color[2]
    icon = cv2.cvtColor(np.array(averages, dtype=np.uint8), cv2.COLOR_BGR2YCR_CB)
    y, cr, cb = cv2.split(icon)
    dct_y = cv2.dct(np.float32(y))
    dct_cb = cv2.dct(np.float32(cb))
    dct_cr = cv2.dct(np.float32(cr))
    dct_y_zigzag = []
    dct_cb_zigzag = []
    dct_cr_zigzag = []
    flip = True
    flipped_dct_y = np.fliplr(dct_y)
    flipped_dct_cb = np.fliplr(dct_cb)
    flipped_dct_cr = np.fliplr(dct_cr)
    for i in range(cld_rows + cld_cols -1):
        k_diag = cld_rows - 1 - i
        diag_y = np.diag(flipped_dct_y, k=k_diag)
        diag_cb = np.diag(flipped_dct_cb, k=k_diag)
        diag_cr = np.diag(flipped_dct_cr, k=k_diag)
        if flip:
            diag_y = diag_y[::-1]
            diag_cb = diag_cb[::-1]
            diag_cr = diag_cr[::-1]
        dct_y_zigzag.append(diag_y)
        dct_cb_zigzag.append(diag_cb)
        dct_cr_zigzag.append(diag_cr)
        flip = not flip
    return np.concatenate([np.concatenate(dct_y_zigzag), np.concatenate(dct_cb_zigzag), np.concatenate(dct_cr_zigzag)])

In [5]:
def extract_ngtdm_features(image_path, level_min=1, level_max=256, threshold=None):
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    extract = NGTDM(image,level_min=1, level_max=256, threshold=1)
    feature = list(extract.features.values())
    return np.array(feature)

In [6]:
images_path = "./fruit_picture"
results = []
names = []
bins = 8
labels = []
global_features = [] 

In [7]:
files = [os.path.join(images_path, p) for p in os.listdir(images_path)]
valid_images = [".jpg",".gif",".png",".tga"]
cnt = 1

glcm = []
lbp = []
hsv = []
colorCorrelogram = []
cld = []
ngtdm = []
names = []

for f in files:
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    print('Extracting features from image number ', cnt , ' : %s' % f)
    label = re.split(r'_+(?=\d)|(?<=\d)_',f.split('/')[-1].lower().split('\\')[1])[0]
    glcm.append(extract_haralick_features(f))
    lbp.append(extract_lbp_features(f))
    hsv.append(extract_histogram_features(f))
    colorCorrelogram.append(extract_correlogram_features(f))
    cld.append(extract_cld_features(f))
    ngtdm.append(extract_ngtdm_features(f))
    
    labels.append(label)
    cnt += 1
    

targetNames = np.unique(labels)
le = LabelEncoder()
target = le.fit_transform(labels)

Extracting features from image number  1  : ./fruit_picture\Banana_1.jpg
Extracting features from image number  2  : ./fruit_picture\Banana_10.jpg
Extracting features from image number  3  : ./fruit_picture\Banana_11.jpg
Extracting features from image number  4  : ./fruit_picture\Banana_12.jpg
Extracting features from image number  5  : ./fruit_picture\Banana_13.jpg
Extracting features from image number  6  : ./fruit_picture\Banana_14.jpg
Extracting features from image number  7  : ./fruit_picture\Banana_15.jpg
Extracting features from image number  8  : ./fruit_picture\Banana_16.jpg
Extracting features from image number  9  : ./fruit_picture\Banana_17.jpg
Extracting features from image number  10  : ./fruit_picture\Banana_18.jpg
Extracting features from image number  11  : ./fruit_picture\Banana_19.jpg
Extracting features from image number  12  : ./fruit_picture\Banana_2.jpg
Extracting features from image number  13  : ./fruit_picture\Banana_20.jpg
Extracting features from image numbe

Extracting features from image number  109  : ./fruit_picture\Cherry_17.jpg
Extracting features from image number  110  : ./fruit_picture\Cherry_18.jpg
Extracting features from image number  111  : ./fruit_picture\Cherry_19.jpg
Extracting features from image number  112  : ./fruit_picture\Cherry_2.jpg
Extracting features from image number  113  : ./fruit_picture\Cherry_20.jpg
Extracting features from image number  114  : ./fruit_picture\Cherry_21.jpg
Extracting features from image number  115  : ./fruit_picture\Cherry_22.jpg
Extracting features from image number  116  : ./fruit_picture\Cherry_23.jpg
Extracting features from image number  117  : ./fruit_picture\Cherry_24.jpg
Extracting features from image number  118  : ./fruit_picture\Cherry_25.jpg
Extracting features from image number  119  : ./fruit_picture\Cherry_26.jpg
Extracting features from image number  120  : ./fruit_picture\Cherry_27.jpg
Extracting features from image number  121  : ./fruit_picture\Cherry_28.jpg
Extracting fe

Extracting features from image number  213  : ./fruit_picture\Dragon_Fruit_20.jpg
Extracting features from image number  214  : ./fruit_picture\Dragon_Fruit_21.jpg
Extracting features from image number  215  : ./fruit_picture\Dragon_Fruit_22.jpg
Extracting features from image number  216  : ./fruit_picture\Dragon_Fruit_23.jpg
Extracting features from image number  217  : ./fruit_picture\Dragon_Fruit_24.jpg
Extracting features from image number  218  : ./fruit_picture\Dragon_Fruit_25.jpg
Extracting features from image number  219  : ./fruit_picture\Dragon_Fruit_26.jpg
Extracting features from image number  220  : ./fruit_picture\Dragon_Fruit_27.jpg
Extracting features from image number  221  : ./fruit_picture\Dragon_Fruit_28.jpg
Extracting features from image number  222  : ./fruit_picture\Dragon_Fruit_29.jpg
Extracting features from image number  223  : ./fruit_picture\Dragon_Fruit_3.jpg
Extracting features from image number  224  : ./fruit_picture\Dragon_Fruit_30.jpg
Extracting featur

Extracting features from image number  314  : ./fruit_picture\Green_Grape_21.jpg
Extracting features from image number  315  : ./fruit_picture\Green_Grape_22.jpg
Extracting features from image number  316  : ./fruit_picture\Green_Grape_23.jpg
Extracting features from image number  317  : ./fruit_picture\Green_Grape_24.jpg
Extracting features from image number  318  : ./fruit_picture\Green_Grape_25.jpg
Extracting features from image number  319  : ./fruit_picture\Green_Grape_26.jpg
Extracting features from image number  320  : ./fruit_picture\Green_Grape_27.jpg
Extracting features from image number  321  : ./fruit_picture\Green_Grape_28.jpg
Extracting features from image number  322  : ./fruit_picture\Green_Grape_29.jpg
Extracting features from image number  323  : ./fruit_picture\Green_Grape_3.jpg
Extracting features from image number  324  : ./fruit_picture\Green_Grape_30.jpg
Extracting features from image number  325  : ./fruit_picture\Green_Grape_31.jpg
Extracting features from imag

Extracting features from image number  421  : ./fruit_picture\Kiwi_28.jpg
Extracting features from image number  422  : ./fruit_picture\Kiwi_29.jpg
Extracting features from image number  423  : ./fruit_picture\Kiwi_3.jpg
Extracting features from image number  424  : ./fruit_picture\Kiwi_30.jpg
Extracting features from image number  425  : ./fruit_picture\Kiwi_31.jpg
Extracting features from image number  426  : ./fruit_picture\Kiwi_32.jpg
Extracting features from image number  427  : ./fruit_picture\Kiwi_33.jpg
Extracting features from image number  428  : ./fruit_picture\Kiwi_34.jpg
Extracting features from image number  429  : ./fruit_picture\Kiwi_35.jpg
Extracting features from image number  430  : ./fruit_picture\Kiwi_36.jpg
Extracting features from image number  431  : ./fruit_picture\Kiwi_37.jpg
Extracting features from image number  432  : ./fruit_picture\Kiwi_38.jpg
Extracting features from image number  433  : ./fruit_picture\Kiwi_39.jpg
Extracting features from image number  

Extracting features from image number  531  : ./fruit_picture\Mango_37.jpg
Extracting features from image number  532  : ./fruit_picture\Mango_38.jpg
Extracting features from image number  533  : ./fruit_picture\Mango_39.jpg
Extracting features from image number  534  : ./fruit_picture\Mango_4.jpg
Extracting features from image number  535  : ./fruit_picture\Mango_40.jpg
Extracting features from image number  536  : ./fruit_picture\Mango_41.jpg
Extracting features from image number  537  : ./fruit_picture\Mango_42.jpg
Extracting features from image number  538  : ./fruit_picture\Mango_43.jpg
Extracting features from image number  539  : ./fruit_picture\Mango_44.jpg
Extracting features from image number  540  : ./fruit_picture\Mango_45.jpg
Extracting features from image number  541  : ./fruit_picture\Mango_46.jpg
Extracting features from image number  542  : ./fruit_picture\Mango_47.jpg
Extracting features from image number  543  : ./fruit_picture\Mango_48.jpg
Extracting features from i

Extracting features from image number  637  : ./fruit_picture\Orange_42.jpg
Extracting features from image number  638  : ./fruit_picture\Orange_43.jpg
Extracting features from image number  639  : ./fruit_picture\Orange_44.jpg
Extracting features from image number  640  : ./fruit_picture\Orange_45.jpg
Extracting features from image number  641  : ./fruit_picture\Orange_46.jpg
Extracting features from image number  642  : ./fruit_picture\Orange_47.jpg
Extracting features from image number  643  : ./fruit_picture\Orange_48.jpg
Extracting features from image number  644  : ./fruit_picture\Orange_49.jpg
Extracting features from image number  645  : ./fruit_picture\Orange_5.jpg
Extracting features from image number  646  : ./fruit_picture\Orange_50.jpg
Extracting features from image number  647  : ./fruit_picture\Orange_6.jpg
Extracting features from image number  648  : ./fruit_picture\Orange_7.jpg
Extracting features from image number  649  : ./fruit_picture\Orange_8.jpg
Extracting featu

Extracting features from image number  740  : ./fruit_picture\Rambutan_45.jpg
Extracting features from image number  741  : ./fruit_picture\Rambutan_46.jpg
Extracting features from image number  742  : ./fruit_picture\Rambutan_47.jpg
Extracting features from image number  743  : ./fruit_picture\Rambutan_48.jpg
Extracting features from image number  744  : ./fruit_picture\Rambutan_49.jpg
Extracting features from image number  745  : ./fruit_picture\Rambutan_5.jpg
Extracting features from image number  746  : ./fruit_picture\Rambutan_50.jpg
Extracting features from image number  747  : ./fruit_picture\Rambutan_6.jpg
Extracting features from image number  748  : ./fruit_picture\Rambutan_7.jpg
Extracting features from image number  749  : ./fruit_picture\Rambutan_8.jpg
Extracting features from image number  750  : ./fruit_picture\Rambutan_9.jpg
Extracting features from image number  751  : ./fruit_picture\Red_Apple_1.jpg
Extracting features from image number  752  : ./fruit_picture\Red_App

Extracting features from image number  844  : ./fruit_picture\Rose_Apple_49.jpg
Extracting features from image number  845  : ./fruit_picture\Rose_Apple_5.jpg
Extracting features from image number  846  : ./fruit_picture\Rose_Apple_50.jpg
Extracting features from image number  847  : ./fruit_picture\Rose_Apple_6.jpg
Extracting features from image number  848  : ./fruit_picture\Rose_Apple_7.jpg
Extracting features from image number  849  : ./fruit_picture\Rose_Apple_8.jpg
Extracting features from image number  850  : ./fruit_picture\Rose_Apple_9.jpg
Extracting features from image number  851  : ./fruit_picture\Salacca_1.jpg
Extracting features from image number  852  : ./fruit_picture\Salacca_10.jpg
Extracting features from image number  853  : ./fruit_picture\Salacca_11.jpg
Extracting features from image number  854  : ./fruit_picture\Salacca_12.jpg
Extracting features from image number  855  : ./fruit_picture\Salacca_13.jpg
Extracting features from image number  856  : ./fruit_picture

Extracting features from image number  949  : ./fruit_picture\Strawberry_8.jpg
Extracting features from image number  950  : ./fruit_picture\Strawberry_9.jpg
Extracting features from image number  951  : ./fruit_picture\Tamarind_1.jpg
Extracting features from image number  952  : ./fruit_picture\Tamarind_10.jpg
Extracting features from image number  953  : ./fruit_picture\Tamarind_11.jpg
Extracting features from image number  954  : ./fruit_picture\Tamarind_12.jpg
Extracting features from image number  955  : ./fruit_picture\Tamarind_13.jpg
Extracting features from image number  956  : ./fruit_picture\Tamarind_14.jpg
Extracting features from image number  957  : ./fruit_picture\Tamarind_15.jpg
Extracting features from image number  958  : ./fruit_picture\Tamarind_16.jpg
Extracting features from image number  959  : ./fruit_picture\Tamarind_17.jpg
Extracting features from image number  960  : ./fruit_picture\Tamarind_18.jpg
Extracting features from image number  961  : ./fruit_picture\T

In [8]:
features = [glcm,
            lbp,
            hsv,
            colorCorrelogram,
            cld,
            ngtdm]
features_name = ['glcm',
                 'lbp',
                 'hsv',
                 'correlogram',
                 'cld',
                 'ngtdm']

scaler = StandardScaler()
features_perm = []
features_name_perm = []

for num in range(1,len(features)+1):
    for f in combinations(features,num):
        feature = np.hstack(list(f))
        combi = scaler.fit_transform(feature)
        features_perm.append(combi)

for num in range(1,len(features)+1):
    for name in combinations(features_name,num):
        features_name_perm.append('_'.join(map(str, name)))

In [9]:
h5f_data = h5py.File('./output/data.h5', 'w')
for i in range(len(features_perm)):
    h5f_data.create_dataset(features_name_perm[i], data=np.array(features_perm[i]))

h5f_label = h5py.File('./output/labels.h5', 'w')
h5f_label.create_dataset('labels', data=np.array(target))

h5f_data.close()
h5f_label.close()

In [10]:
# print(h5f_data)
# print(len(features_name_perm))
targetNames

array(['banana', 'blueberry', 'cherry', 'chinese_pear', 'dragon_fruit',
       'green_apple', 'green_grape', 'guava', 'kiwi', 'longan', 'mango',
       'marian_plum', 'orange', 'passion_fruit', 'rambutan', 'red_apple',
       'rose_apple', 'salacca', 'strawberry', 'tamarind'], dtype='<U13')